In [1]:
import jax.numpy as jnp
from jax import random
from jax import lax as jlax
from jax.tree_util import register_pytree_node_class

In [42]:
@register_pytree_node_class
class Parameter:
    """
    A class to represent a parameter with a name and a value, supporting basic arithmetic operations
    and integration with the JAX pytree system.

    Attributes:
    -----------
    name : str
        The name of the parameter.
    value : ndarray
        The value of the parameter, typically a NumPy array or similar.
    shape : tuple
        The shape of the value.
    """
    def __init__(self,name,value):
        self.name = name
        self.value = value
        self.shape = value.shape
    def __sub__(self,param):
        if isinstance(param,Parameter):
            return Parameter(self.name,self.value-param.value)
        raise TypeError(f"unsupported operand type(s) for -: {type(param)} and 'Parameter'")
    def __add__(self,other):
        if isinstance(other,Parameter):
            return Parameter(self.name,self.value+other.value)
        if isinstance(other,float):
            return Parameter(self.name,self.value+other)
        raise TypeError(f"unsupported operand type(s) for +: {type(other)} and 'Parameter'")
    def __mul__(self,factor):
        if isinstance(factor,float):
            return Parameter(self.name,factor*self.value)
        raise TypeError(f'Cannot multiply a Parameter with {type(factor)}')
    def __rmul__(self,factor):
        if isinstance(factor,float):
            return Parameter(self.name,factor*self.value)
        raise TypeError(f'Cannot multiply a Parameter with {type(factor)}')
    def __pow__(self,factor):
        return Parameter(self.name,self.value**factor)
    def __truediv__(self,other):
        if isinstance(other,Parameter):
            return Parameter(self.name,self.value/other.value)
        if isinstance(other,float):
            return Parameter(self.name,self.value/other)
        raise TypeError(f'Cannot divide a Parameter with {type(other)}')
    def tree_flatten(self):
        children = (self.value,)
        aux_data = (self.name,)
        return (children, aux_data)
    @classmethod
    def tree_unflatten(cls, aux_data, children):
        return cls(*aux_data,*children)

In [3]:
@register_pytree_node_class
class LinearParams:
    """
    A class to represent the parameters of a linear layer, specifically the weights, which are stored
    as a Parameter object.

    Attributes:
    -----------
    name : str
        The name of the linear layer parameters.
    weights : Parameter
        The weights of the linear layer, stored as a Parameter object.
    """
    def __init__(self,name,weights):
        self.name = name
        if isinstance(weights,Parameter):
            self.weights = weights
        else:
            self.weights = Parameter("W",weights)
    def __sub__(self,other):
        if isinstance(other,LinearParams):
            return LinearParams(self.name,self.weights-other.weights)
        raise TypeError(f"unsupported operand type(s) for -: {type(other)} and 'LinearParams'")
    def __add__(self,other):
        if isinstance(other,LinearParams) :
            return LinearParams(self.name,self.weights+other.weights)
        if isinstance(other,float) :
            return LinearParams(self.name,self.weights+other)
        raise TypeError(f"unsupported operand type(s) for +: {type(other)} and 'LinearParams'")
    def __mul__(self,other):
        if isinstance(other,float):
            return LinearParams(self.name,self.weights*other)
        raise TypeError(f"Cannot multiply a 'LinearParams' with {type(other)}")
    def __rmul__(self,other):
        if isinstance(other,float):
            return LinearParams(self.name,self.weights*other)
        raise TypeError(f"Cannot multiply a 'LinearParams' with {type(other)}")
    def __truediv__(self,other):
        if isinstance(other,LinearParams) :
            return LinearParams(self.name,self.weights/other.weights)
        if isinstance(other,float):
            return LinearParams(self.name,self.weights/other)
        raise TypeError(f"Cannot divide a 'LinearParams' with {type(other)}")
    def __pow__(self,factor):
        return LinearParams(self.name,self.weights**factor)
    def tree_flatten(self):
        children = (self.weights,)
        aux_data = (self.name,)
        return (children, aux_data)
    @classmethod
    def tree_unflatten(cls, aux_data, children):
        return cls(*aux_data,*children)

In [4]:
@register_pytree_node_class
class FeedForwardParams:
    """
    A class to represent the parameters of a feedforward neural network layer, specifically the weights and biases,
    which are stored as Parameter objects.

    Attributes:
    -----------
    name : str
        The name of the feedforward layer parameters.
    weights : Parameter
        The weights of the feedforward layer, stored as a Parameter object.
    bias : Parameter
        The bias of the feedforward layer, stored as a Parameter object.
    """
    def __init__(self,name,weights,bias):
        self.name = name
        if isinstance(weights,Parameter):
            self.weights = weights
        else:
            self.weights = Parameter("W",weights)
        if isinstance(bias,Parameter):
            self.bias = bias
        else:
            self.bias = Parameter("bais",bias)
    def __sub__(self,other):
        if isinstance(other,FeedForwardParams) :
            return FeedForwardParams(self.name,self.weights-other.weights,self.bias-other.bias)
        raise TypeError(f"unsupported operand type(s) for -: {type(other)} and 'FeedForwardParams'")
    def __add__(self,other):
        if isinstance(other,FeedForwardParams):
            return FeedForwardParams(self.name,self.weights+other.weights,self.bias+other.bias)
        if isinstance(other,float):
            return FeedForwardParams(self.name,self.weights+other,self.bias+other)
        raise TypeError(f"unsupported operand type(s) for +: {type(other)} and 'FeedForwardParams'")
    def __mul__(self,other):
        if isinstance(other,float):
            return FeedForwardParams(self.name,self.weights*other,self.bias*other)
        raise TypeError(f"Cannot multiply a 'FeedForwardParams' with {type(other)}")
    def __rmul__(self,other):
        if isinstance(other,float):
            return FeedForwardParams(self.name,self.weights*other,self.bias*other)
        raise TypeError(f"Cannot multiply a 'FeedForwardParams' with {type(other)}")
    def __truediv__(self,other):
        if isinstance(other,FeedForwardParams):
            return FeedForwardParams(self.name,self.weights/other.weights,self.bias/other.bias)
        if isinstance(other,float):
            return FeedForwardParams(self.name,self.weights/other,self.bias/other)
        raise TypeError(f"Cannot divide a 'FeedForwardParams' with {type(other)}")
    def __pow__(self,factor):
        return FeedForwardParams(self.name,self.weights**factor,self.bias**factor)
    def tree_flatten(self):
        children = (self.weights,self.bias,)
        aux_data = (self.name,)
        return (children, aux_data)
    @classmethod
    def tree_unflatten(cls, aux_data, children):
        return cls(*aux_data,*children)
    

In [5]:
@register_pytree_node_class
class AttentionParams:
    """
    A class to represent the parameters of an attention mechanism, specifically the weights for key, query, and value,
    which are stored as Parameter objects.

    Attributes:
    -----------
    name : str
        The name of the attention parameters.
    w_k : Parameter
        The weights for the key, stored as a Parameter object.
    w_q : Parameter
        The weights for the query, stored as a Parameter object.
    w_v : Parameter
        The weights for the value, stored as a Parameter object.
    """
    def __init__(self,name,w_k,w_q,w_v):
        self.name = name
        if isinstance(w_k,Parameter):
            self.w_k = w_k
        else:
            self.w_k = Parameter("w_k",w_k)
        if isinstance(w_q,Parameter):
            self.w_q = w_q
        else:
            self.w_q = Parameter("w_q",w_q)
        if isinstance(w_v,Parameter):
            self.w_v = w_v
        else:
            self.w_v = Parameter("w_v",w_v)
        
    def __sub__(self,other):
        if isinstance(other,AttentionParams):
            return AttentionParams(self.name,self.w_k-other.w_k,self.w_q-other.w_q,self.w_v-other.w_v)
        raise TypeError(f"unsupported operand type(s) for -: {type(other)} and 'AttentionParams'")
    def __add__(self,other):
        if isinstance(other,AttentionParams):
            return AttentionParams(self.name,self.w_k+other.w_k,self.w_q+other.w_q,self.w_v+other.w_v)
        if isinstance(other,float):
            return AttentionParams(self.name,self.w_k+other,self.w_q+other,self.w_v+other)
        raise TypeError(f"unsupported operand type(s) for +: {type(other)} and 'AttentionParams'")
    def __mul__(self,other):
        if isinstance(other,float):
            return AttentionParams(self.name,self.w_k*other,self.w_q*other,self.w_v*other)
        raise TypeError(f"Cannot multiply a 'AttentionParams' with {type(other)}")
    def __truediv__(self,other):
        if isinstance(other,AttentionParams):
            return AttentionParams(self.name,self.w_k/other.w_k,self.w_q/other.w_q,self.w_v/other.w_v)
        if isinstance(other,float):
            return AttentionParams(self.name,self.w_k/other,self.w_q/other,self.w_v/other)
        raise TypeError(f"Cannot divide a 'AttentionParams' with {type(other)}")
    def __rmul__(self,other):
        if isinstance(other,float):
            return AttentionParams(self.name,self.w_k*other,self.w_q*other,self.w_v*other)
        raise TypeError(f"Cannot multiply a 'AttentionParams' with {type(other)}")
    def __pow__(self,factor):
        return AttentionParams(self.name,self.w_k**factor,self.w_q**factor,self.w_v**factor)
    def tree_flatten(self):
        children = (self.w_k,self.w_q,self.w_v,)
        aux_data = (self.name,)
        return (children, aux_data)
    @classmethod
    def tree_unflatten(cls, aux_data, children):
        return cls(*aux_data,*children)

In [6]:
@register_pytree_node_class
class MultiHeadAttentionParams:
    """
    A class to represent the parameters of a multi-head attention mechanism, including the output weights
    and a list of individual attention heads.

    Attributes:
    -----------
    name : str
        The name of the multi-head attention parameters.
    weights : Parameter
        The output weights of the multi-head attention, stored as a Parameter object.
    heads : list[AttentionParams]
        A list of AttentionParams objects representing the individual attention heads.
    num_heads : int
        The number of attention heads.
    """
    def __init__(self,name,weights,heads:list[AttentionParams]):
        self.name = name
        if isinstance(weights,Parameter):
            self.weights = weights
        else:
            self.weights = Parameter("Wo",weights)
        self.heads = heads
        self.num_heads = len(heads)
    def add(self,head1,head2):
        return head1+head2
    def subtract(self,head1,head2):
        return head1-head2
    def multiply(self,val,head):
        return val*head
    def divide(self,head,val):
        return head/val
    def pow(self,val,head):
        return head**val
    def __sub__(self,other):
        if isinstance(other,MultiHeadAttentionParams):
            heads = list(map(self.subtract,self.heads,other.heads))
            return MultiHeadAttentionParams(self.name,self.weights-other.weights,heads)
        raise TypeError(f"unsupported operand type(s) for -: {type(other)} and 'MultiHeadAttentionParams'")
    def __add__(self,other):
        if isinstance(other,MultiHeadAttentionParams) :
            heads = list(map(self.add,self.heads,other.heads))
            return MultiHeadAttentionParams(self.name,self.weights+other.weights,heads)
        if isinstance(other,float):
            heads = list(map(self.add,self.heads,[other]*self.num_heads))
            return MultiHeadAttentionParams(self.name,self.weights+other,heads)
        raise TypeError(f"unsupported operand type(s) for +: {type(other)} and 'MultiHeadAttentionParams'")
    def __mul__(self,other):
        if isinstance(other,float):
            heads = list(map(self.multiply,[other]*self.num_heads,self.heads))
            return MultiHeadAttentionParams(self.name,self.weights*other,heads)
        raise TypeError(f"Cannot multiply a 'MultiHeadAttentionParams' with {type(other)}")
    def __truediv__(self,other):
        if isinstance(other,MultiHeadAttentionParams) :
            heads = list(map(self.divide,self.heads,other.heads))
            return MultiHeadAttentionParams(self.name,self.weights/other.weights,heads)
        if isinstance(other,float):
            heads = list(map(self.divide,self.heads,[other]*self.num_heads))
            return MultiHeadAttentionParams(self.name,self.weights/other,heads)
        raise TypeError(f"Cannot multiply a 'MultiHeadAttentionParams' with {type(other)}")
    def __rmul__(self,other):
        if isinstance(other,float):
            heads = list(map(self.multiply,[other]*self.num_heads,self.heads))
            return MultiHeadAttentionParams(self.name,self.weights*other,heads)
        raise TypeError(f"Cannot multiply a 'MultiHeadAttentionParams' with {type(other)}")
    def __pow__(self,factor):
        heads = list(map(self.pow,[factor]*self.num_heads,self.heads))
        return MultiHeadAttentionParams(self.name,self.weights**factor,heads)
    def tree_flatten(self):
        children = (self.weights,self.heads,)
        aux_data = (self.name,)
        return (children, aux_data)
    @classmethod
    def tree_unflatten(cls, aux_data, children):
        return cls(*aux_data,*children)

In [7]:
@register_pytree_node_class
class ModuleParams:
    """
    A class to represent the parameters of a module, including a list of its components.

    Attributes:
    -----------
    name : str
        The name of the module parameters.
    components : list
        A list of components that make up the module.
    num_comps : int
        The number of components in the module.
    """
    def __init__(self,name,components):
        self.name = name
        self.components = components
        self.num_comps = len(components)
    def multiply(self,val,comp):
        return val*comp
    def subtract(self,comp1,comp2):
        return comp1-comp2
    def add(self,comp1,comp2):
        return comp1+comp2
    def pow(self,val,comp):
        return comp**val
    def divide(self,comp,val):
        return comp/val
    def __sub__(self,other):
        if isinstance(other,ModuleParams):
            comps = list(map(self.subtract,self.components,other.components))
            return ModuleParams(self.name,comps)
        raise TypeError(f"unsupported operand type(s) for -: {type(other)} and 'ModuleParams'")
    def __add__(self,other):
        if isinstance(other,ModuleParams) :
            comps = list(map(self.add,self.components,other.components))
            return ModuleParams(self.name,comps)
        if isinstance(other,float):
            comps = list(map(self.add,self.components,[other]*self.num_comps))
            return ModuleParams(self.name,comps)
        raise TypeError(f"unsupported operand type(s) for +: {type(other)} and 'ModuleParams'")
    def __mul__(self,other):
        if isinstance(other,float):
            comps = list(map(self.multiply,[other]*self.num_comps,self.components))
            return ModuleParams(self.name,comps)
        raise TypeError(f"Cannot multiply a 'ModuleParams' with {type(other)}")
    def __truediv__(self,other):
        if isinstance(other,ModuleParams) :
            comps = list(map(self.divide,self.components,other.components))
            return ModuleParams(self.name,comps)
        if isinstance(other,float):
            comps = list(map(self.divide,self.components,[other]*self.num_comps))
            return ModuleParams(self.name,comps)
        raise TypeError(f"Cannot divide a 'ModuleParams' with {type(other)}")
    def __rmul__(self,other):
        if isinstance(other,float):
            comps = list(map(self.multiply,[other]*self.num_comps,self.components))
            return ModuleParams(self.name,comps)
        raise TypeError(f"Cannot multiply a 'ModuleParams' with {type(other)}")
    def __pow__(self,factor):
        comps = list(map(self.pow,[factor]*self.num_comps,self.components))
        return ModuleParams(self.name,comps)
    def tree_flatten(self):
        children = (self.components,)
        aux_data = (self.name,)
        return (children, aux_data)
    @classmethod
    def tree_unflatten(cls, aux_data, children):
        return cls(*aux_data,*children)

In [43]:
weights1 = Parameter("W",random.normal(random.key(29),shape=(24,16)))

In [44]:
weights2 = Parameter("dw",random.normal(random.key(29),shape=(24,16)))

In [45]:
weights3 = (weights1 - 0.1*weights2)

In [51]:
isinstance(weights2,Parameter)

True

In [53]:
(weights3/weights1).value

Array([[0.8999999 , 0.90000004, 0.90000004, 0.9       , 0.9       ,
        0.9       , 0.90000004, 0.90000004, 0.9       , 0.90000004,
        0.9       , 0.90000004, 0.9       , 0.9       , 0.9       ,
        0.9       ],
       [0.90000004, 0.9       , 0.90000004, 0.9       , 0.9       ,
        0.9       , 0.90000004, 0.9       , 0.9       , 0.9       ,
        0.9       , 0.9       , 0.90000004, 0.90000004, 0.90000004,
        0.90000004],
       [0.90000004, 0.90000004, 0.90000004, 0.90000004, 0.8999999 ,
        0.90000004, 0.9000001 , 0.90000004, 0.9000001 , 0.9       ,
        0.90000004, 0.9       , 0.9       , 0.9       , 0.9000001 ,
        0.8999999 ],
       [0.90000004, 0.90000004, 0.9000001 , 0.9000001 , 0.90000004,
        0.9       , 0.9       , 0.90000004, 0.90000004, 0.9       ,
        0.90000004, 0.90000004, 0.9       , 0.9       , 0.90000004,
        0.9       ],
       [0.9       , 0.90000004, 0.9       , 0.9       , 0.90000004,
        0.90000004, 0.90000004, 